<a id='start'></a>
# Анализ сервиса вопросов и ответов по программированию

Проект состоит из двух частей.

В каждой части необходимо выполнить несколько задач в SQL-тренажёре и сформулировать вывод на основе выявленной информации.

## Содержание проекта

1. [Описание данных](#part1)
2. [Часть 1](#part2)
3. [Часть 2](#part3)

<a id='part1'></a>
## Описание данных

**Таблица `stackoverflow.badges`**

Хранит информацию о значках, которые присуждаются за разные достижения. Например, пользователь, правильно ответивший на большое количество вопросов про PostgreSQL, может получить значок postgresql.

|Поле|Описание|
|----|--------|
|id|Идентификатор значка, первичный ключ таблицы|
|name|Название значка|
|user_id|Идентификатор пользователя, которому присвоили значок, внешний ключ, отсылающий к таблице `users`|
|creation_date|Дата присвоения значка|

**Таблица `stackoverflow.post_types`**

Содержит информацию о типе постов. Их может быть два:
- `Question` — пост с вопросом;
- `Answer` — пост с ответом.

|Поле|Описание|
|----|--------|
|id|Идентификатор типа поста, первичный ключ таблицы|
|type|Тип поста|

**Таблица `stackoverflow.posts`**

Содержит информацию о постах.

|Поле|Описание|
|----|--------|
|id|Идентификатор поста, первичный ключ таблицы|
|title|Заголовок поста|
|creation_date|Дата создания поста|
|favorites_count|Число, которое показывает, сколько раз пост добавили в «Закладки»|
|last_activity_date|Дата последнего действия в посте, например комментария|
|last_edit_date|Дата последнего изменения поста|
|user_id|Идентификатор пользователя, который создал пост, внешний ключ к таблице `users`|
|parent_id|Если пост написали в ответ на другую публикацию, в это поле попадёт идентификатор поста с вопросом|
|post_type_id|Идентификатор типа поста, внешний ключ к таблице `post_types`|
|score|Количество очков, которое набрал пост|
|views_count|Количество просмотров|

**Таблица `stackoverflow.users`**

Содержит информацию о пользователях.

|Поле|Описание|
|----|--------|
|id|Идентификатор пользователя, первичный ключ таблицы|
|creation_date|Дата регистрации пользователя|
|display_name|Имя пользователя|
|last_access_date|Дата последнего входа|
|location|Местоположение|
|reputation|Очки репутации, которые получают за хорошие вопросы и полезные ответы|
|views|Число просмотров профиля пользователя|

**Таблица `stackoverflow.vote_types`**

Содержит информацию о типах голосов. Голос — это метка, которую пользователи ставят посту. Типов бывает несколько: 
- `UpMod` — такую отметку получают посты с вопросами или ответами, которые пользователи посчитали уместными и полезными.
- `DownMod` — такую отметку получают посты, которые показались пользователям наименее полезными.
- `Close` — такую метку ставят опытные пользователи сервиса, если заданный вопрос нужно доработать или он вообще не подходит для платформы.
- `Offensive` — такую метку могут поставить, если пользователь ответил на вопрос в грубой и оскорбительной манере, например, указав на неопытность автора поста.
- `Spam` — такую метку ставят в случае, если пост пользователя выглядит откровенной рекламой.

|Поле|Описание|
|----|--------|
|id|Идентификатор типа голоса, первичный ключ|
|name|Название метки|

**Таблица `stackoverflow.votes`**

Содержит информацию о голосах за посты.

|Поле|Описание|
|----|--------|
|id|Идентификатор голоса, первичный ключ|
|post_id|Идентификатор поста, внешний ключ к таблице `posts`|
|user_id|Идентификатор пользователя, который поставил посту голос, внешний ключ к таблице `users`|
|bounty_amount|Сумма вознаграждения, которое назначают, чтобы привлечь внимание к посту|
|vote_type_id|Идентификатор типа голоса, внешний ключ к таблице `vote_types`|
|creation_date|Дата назначения голоса|

<a id='part2'></a>
## Часть 1

### Задание 1
Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

Запрос:
```sql
    SELECT count(id)
    FROM stackoverflow.posts
    WHERE (score > 300 OR favorites_count >= 100) AND post_type_id = '1'
```

### Задание 2
Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

Запрос:
```sql
WITH day_posts_count AS (
SELECT COUNT(id) AS posts_count
FROM stackoverflow.posts
WHERE post_type_id = 1 AND (creation_date::date BETWEEN '2008-11-01' AND '2008-11-18')
GROUP BY creation_date::date
    )

SELECT ROUND(AVG(posts_count), 0)
FROM day_posts_count
```

### Задание 3
Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

```sql
WITH users AS (
SELECT u.id AS user_ids
FROM stackoverflow.users u
JOIN stackoverflow.badges b ON u.id = b.user_id
WHERE u.creation_date::date = b.creation_date::date)

SELECT COUNT(DISTINCT user_ids)
FROM users
```

### Задание 4
Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

```sql
WITH user_posts
AS (
SELECT p.id AS post_ids
FROM stackoverflow.users u
JOIN stackoverflow.posts p ON p.user_id = u.id
WHERE u.display_name = 'Joel Coehoorn')

SELECT COUNT( DISTINCT post_ids)
FROM user_posts
JOIN stackoverflow.votes v ON v.post_id = user_posts.post_ids
```

### Задание 5
Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

```sql
SELECT *,
       ROW_NUMBER() OVER (ORDER BY id DESC)
FROM stackoverflow.vote_types
ORDER BY id
```

### Задание 6
Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

```sql
SELECT user_id, COUNT(id) AS count
FROM stackoverflow.votes
WHERE vote_type_id = 6
GROUP BY user_id
ORDER BY count DESC, user_id DESC
LIMIT 10
```

### Задание 7
Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

```sql
WITH badges_count_table AS (
SELECT DISTINCT user_id, COUNT(id) AS badges_count
FROM stackoverflow.badges
WHERE creation_date::date BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY user_id
ORDER BY badges_count DESC),

ranking AS (
SELECT *,
       DENSE_RANK() OVER (ORDER BY badges_count DESC) AS rank
FROM badges_count_table
ORDER BY badges_count DESC, user_id)

SELECT *
FROM ranking
ORDER BY badges_count DESC, user_id
LIMIT 10
```

### Задание 8
Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

```sql
SELECT title, user_id, score,
       ROUND(AVG(score) OVER (PARTITION BY user_id), 0)
FROM stackoverflow.posts
WHERE title IS NOT NULL AND score != 0
```

### Задание 9
Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

```sql
WITH over_1000 AS (
SELECT user_id
FROM stackoverflow.badges
GROUP BY user_id
HAVING COUNT(id) > 1000)

SELECT title
FROM stackoverflow.posts p
JOIN over_1000 ON over_1000.user_id = p.user_id
WHERE title IS NOT NULL
```

### Задание 10
Напишите запрос, который выгрузит данные о пользователях из Канады (англ. Canada). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с количеством просмотров меньше либо равным нулю не должны войти в итоговую таблицу.

```sql
SELECT id AS user_id, views,
       CASE
           WHEN views >= 350 THEN 1
           WHEN views < 350 AND views >= 100 THEN 2
           ELSE 3
       END AS category
FROM stackoverflow.users
WHERE location LIKE '%Canada%' AND views > 0
```

### Задание 11
Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

```sql
WITH categories AS
(SELECT id AS user_id, views,
       CASE
           WHEN views >= 350 THEN 1
           WHEN views < 350 AND views >= 100 THEN 2
           ELSE 3
       END AS category
FROM stackoverflow.users
WHERE location LIKE '%Canada%' AND views > 0),

leaders AS (
    SELECT *,
       MAX(views) OVER (PARTITION BY category)
    FROM categories)

SELECT user_id, category, views
FROM leaders
WHERE views = max
ORDER BY views DESC, user_id
```

### Задание 12
Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

```sql
WITH user_count AS (
SELECT EXTRACT(DAY FROM creation_date::date) AS day,
       COUNT(id) AS user_count
FROM stackoverflow.users
WHERE EXTRACT(YEAR FROM creation_date::date) = 2008 AND
      EXTRACT(MONTH FROM creation_date::date) = 11
GROUP BY day)

SELECT *,
       SUM(user_count) OVER (ORDER BY day)
FROM user_count
```

### Задание 13
Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

```sql
WITH user_posts AS (
SELECT DISTINCT user_id, creation_date AS first_post,
       ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY creation_date) AS rank
FROM stackoverflow.posts
ORDER BY user_id)

SELECT user_id, first_post - u.creation_date
FROM user_posts
JOIN stackoverflow.users u ON u.id = user_posts.user_id
WHERE rank = 1
```

<a id='part3'></a>
## Часть 2

### Задание 1
Выведите общую сумму просмотров у постов, опубликованных в каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

```sql
SELECT DISTINCT DATE_TRUNC('month', creation_date)::date AS month,
       SUM(views_count) OVER (PARTITION BY DATE_TRUNC('month', creation_date)::date) AS sum_views
FROM stackoverflow.posts
WHERE EXTRACT(YEAR FROM creation_date::date) = 2008
ORDER BY sum_views DESC
```

### Задание 2
Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

```sql
SELECT u.display_name,
       COUNT(DISTINCT p.user_id)
FROM stackoverflow.users AS u
JOIN stackoverflow.posts AS p ON u.id = p.user_id
JOIN stackoverflow.post_types AS p_t ON p.post_type_id = p_t.id
WHERE p_t.id = 2 AND (DATE_TRUNC('day', p.creation_date)::date <= DATE_TRUNC('day', u.creation_date)::date + INTERVAL '1 months')
GROUP BY u.display_name
HAVING COUNT(DISTINCT p.id) > 100
ORDER BY u.display_name
```

### Задание 3
Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

```sql
WITH users_09_2008 AS (
SELECT u.id AS user_id, p.id AS post_id, p.creation_date AS post_dt
FROM stackoverflow.users u
    JOIN stackoverflow.posts p ON p.user_id = u.id
WHERE EXTRACT(YEAR FROM u.creation_date) = 2008 AND
      EXTRACT(MONTH FROM u.creation_date) = 9),
      
posts_12_2008 AS (
SELECT user_id
FROM users_09_2008
WHERE EXTRACT(MONTH FROM post_dt) = 12)

SELECT DATE_TRUNC('month', post_dt)::date AS month, COUNT(DISTINCT post_id)
FROM users_09_2008
JOIN posts_12_2008 ON users_09_2008.user_id = posts_12_2008.user_id
GROUP BY month
ORDER BY month DESC
```

### Задание 4
Используя данные о постах, выведите несколько полей:
- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумма просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

```sql
SELECT user_id, creation_date, views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts
ORDER BY user_id, creation_date
```

### Задание 5
Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

```sql
WITH table_1 AS (
SELECT id, user_id, creation_date::date
FROM stackoverflow.posts
WHERE creation_date::date BETWEEN '2008-12-01' AND '2008-12-07'),

table_2 AS (
SELECT user_id, COUNT(DISTINCT creation_date) AS days_count
FROM table_1
GROUP BY user_id)

SELECT ROUND(AVG(days_count), 0)
FROM table_2
```

### Задание 6
На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
- Номер месяца.
- Количество постов за месяц.
- Процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.

Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

```sql
WITH table_1 AS (
SELECT EXTRACT(MONTH FROM creation_date) AS month,
       COUNT(DISTINCT id) AS post_count
FROM stackoverflow.posts
WHERE creation_date BETWEEN '2008-09-01' AND '2008-12-31'
GROUP BY month)

SELECT *,
       ROUND((((post_count - LAG(post_count) OVER (ORDER BY month)))/LAG(post_count) OVER (ORDER BY month)::numeric)*100, 2) AS diff
FROM table_1
ORDER BY month
```

### Задание 7
Найдите пользователя, который опубликовал больше всего постов за всё время с момента регистрации. Выведите данные его активности за октябрь 2008 года в таком виде:
- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

```sql
WITH most_active_user AS (
SELECT user_id, COUNT(DISTINCT id)
FROM stackoverflow.posts
GROUP BY user_id
ORDER BY count DESC
LIMIT 1),

table_2 AS (
SELECT EXTRACT(WEEK FROM creation_date) AS week, creation_date AS post_dt,
       ROW_NUMBER() OVER (PARTITION BY EXTRACT(WEEK FROM creation_date) ORDER BY creation_date)
FROM stackoverflow.posts p
JOIN most_active_user ON p.user_id = most_active_user.user_id
WHERE creation_date::date BETWEEN '2008-10-01' AND '2008-10-31'),

table_3 AS (
SELECT *,
       MAX(row_number) OVER (PARTITION BY week) AS last_post
FROM table_2)

SELECT week, post_dt
FROM table_3
WHERE row_number = last_post
```

**[В начало](#start)**